# ColabRTC examples
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/thefonseca/colabrtc/blob/master/examples/colabrtc.ipynb)

This notebook show how stream images from a camera on localhost to a remote Colaboratory session. The API allows one to specify an arbitrary frame processing routine that can leverage Colab resources.

## Setting up the environment

In [0]:
!git clone https://github.com/thefonseca/colabrtc.git
!chmod +x ./colabrtc/install.sh && ./colabrtc/install.sh

import sys
sys.path.insert(0, "/content/colabrtc/colabrtc")

## A simple echo call
In this simple example the Colab peer will just return the received frames.

In [5]:
from call import ColabCall

call = ColabCall()
call.create()

# One could also specify a signaling folder on Google Drive and establish 
# signaling between different Colab instances (or other apps with access to 
# your Google Drive)
# call.create(signaling_folder='/content/drive/My Drive/webrtc/rooms')
call.join()

<IPython.core.display.Javascript object>

INFO:colabrtc.signaling:Room ID: 7003220860
INFO:colabrtc.signaling:Peer ID: 3563152480
INFO:colabrtc.signaling:Room ID: 7003220860
INFO:colabrtc.signaling:Peer ID: 4470233947


## Processing frames with OpenCV



In [6]:
import cv2
def process_frame(frame, frame_idx):
  if frame_idx % 2 == 1:
    edges = cv2.Canny(frame, 100, 200)
    return cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
  
call = ColabCall()
call.create(frame_transformer=process_frame)
call.join()

<IPython.core.display.Javascript object>

INFO:colabrtc.signaling:Room ID: 6122042301
INFO:colabrtc.signaling:Peer ID: 3695546120
INFO:colabrtc.signaling:Room ID: 6122042301
INFO:colabrtc.signaling:Peer ID: 0999196252


## Using a model that requires a GPU

One motivation for this project is to leverage a remote GPU testing and prototyping. In this example, we show how to use a [custom FrameTransformer](https://github.com/thefonseca/colabrtc/blob/master/colabrtc/examples/avatarify_colab.py) to [Avatarify](https://github.com/alievk/avatarify) yourself (performance is still poor on the free K80 GPUs, and the WebRTC connection eventually breaks).

In [7]:
# Make sure you have a GPU accelerator enabled
!chmod +x ./colabrtc/examples/install_avatarify.sh && ./colabrtc/examples/install_avatarify.sh

Cloning repository https://github.com/alievk/avatarify.git...
Installing FOMM submodule...
Avatarify setup complete!


In [16]:
import random
room = "".join([random.choice("0123456789") for x in range(10)])
print('Starting call in room', room)

# Due to multiprocessing support limitations, we need to run the Python peer via commandline.
!nohup python3 /content/colabrtc/examples/avatarify_colab.py --room $room > nohup.txt 2>&1 &

Starting call in room 2277410356


In [17]:
# Check out the nohup.txt file contents before running this cell
# You should see at the end something like (it may take a while):
# INFO:colabrtc.signaling:Room ID: 0065888135
# INFO:colabrtc.signaling:Peer ID: 9336811461

from call import ColabCall
call = ColabCall()
call.join(room=room)

<IPython.core.display.Javascript object>

INFO:colabrtc.signaling:Room ID: 2277410356
INFO:colabrtc.signaling:Peer ID: 9798329600


This is one easy way to kill the background process.

In [15]:
!pkill -f avatarify_colab.py
!ps aux | grep avatarify_colab.py

root        1443  0.0  0.0  39196  6520 ?        S    20:43   0:00 /bin/bash -c ps aux | grep avatarify_colab.py
root        1445  0.0  0.0  38568  5488 ?        S    20:43   0:00 grep avatarify_colab.py
